<a href="https://colab.research.google.com/github/p-ai-org/p-web/blob/main/pai_web.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!git clone https://github.com/p-ai-org/p-web.git

Cloning into 'p-web'...
remote: Enumerating objects: 72977, done.
remote: Counting objects: 100% (9116/9116), done.
remote: Compressing objects: 100% (425/425), done.
remote: Total 72977 (delta 8693), reused 9110 (delta 8689), pack-reused 63861
Receiving objects: 100% (72977/72977), 833.92 MiB | 16.02 MiB/s, done.
Resolving deltas: 100% (22259/22259), done.
Checking out files: 100% (78143/78143), done.


In [2]:
!pip install transformers
!pip install tokenizers

     |████████████████████████████████| 2.1MB 7.1MB/s 
     |████████████████████████████████| 3.3MB 34.6MB/s 
     |████████████████████████████████| 901kB 34.8MB/s 


In [5]:
from tokenizers.implementations import ByteLevelBPETokenizer

tokenizer = ByteLevelBPETokenizer("p-web/tokenize/vocab.json","p-web/tokenize/merges.txt",)

!mkdir BPETOK
tokenizer.save_model("BPETOK")

['BPETOK/vocab.json', 'BPETOK/merges.txt']

In [6]:
tokenizer.encode("<!DOCTYPE html><html><body><h1>My First Heading</h1><p>My first paragraph.</p></body></html>").tokens

['<',
 '!',
 'D',
 'O',
 'C',
 'T',
 'Y',
 'P',
 'E',
 'html',
 '>',
 '<',
 'html',
 '>',
 '<',
 'body',
 '>',
 '<',
 'h',
 '1',
 '>',
 'M',
 'y',
 'F',
 'ir',
 'st',
 'H',
 'ea',
 'ding',
 '</',
 'h',
 '1',
 '>',
 '<',
 'p',
 '>',
 'M',
 'y',
 'first',
 'paragraph',
 '.',
 '</',
 'p',
 '>',
 '</',
 'body',
 '>',
 '</',
 'html',
 '>']

In [7]:
from ast import literal_eval
import random
import os

trainData = ''
testData = ''
path = "/content/p-web/tokenize/pre-tokenized"
counter = 0
for file in os.scandir(path):
  counter += 1
  if counter > 1000: break
  sourceFile = os.path.join(path, file.name)
  with open(sourceFile, 'r', encoding="ISO-8859-1") as f:
    try:
      ptstr = f.read() # get pretokens as a string
    except Exception as e:
      print('During file reading, this error occurred:', e)
    
    if random.random() < 0.15:
      testData += ptstr
    else:
      trainData += ptstr
    
trainf = open("train_data1.txt", 'w')
trainf.write(trainData)
trainf.close()
testf = open("test_data1.txt",'w')
testf.write(testData)
testf.close()


  

In [8]:
train_path = 'train_data1.txt'
test_path = 'test_data1.txt'

In [9]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("BPETOK")


In [13]:

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [14]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer): 
  train_dataset = TextDataset( 
      tokenizer=tokenizer, 
      file_path=train_path, 
      block_size=128)
  
  test_dataset = TextDataset(
      tokenizer=tokenizer,
      file_path=test_path,
      block_size=128)   
 
  data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False)
  
  return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


In [15]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("anonymous-german-nlp/german-gpt2")


training_args = TrainingArguments(
    output_dir="./gpt2-gerchef", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:762: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [8]:
from transformers import Trainer, TrainingArguments,AutoModelForCausalLM, AutoConfig

config = AutoConfig.from_pretrained("gpt2-large")
model = AutoModelForCausalLM.from_config(config)


training_args = TrainingArguments(
    output_dir="gpt2_html", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [16]:
trainer.train()

TypeError: ignored